In [5]:
%tensorflow_version 2.x

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

TensorFlow 2.x selected.


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
order = np.load('drive/My Drive/orderings.npy')
distances = np.load('drive/My Drive/distances.npy')
social = np.load('drive/My Drive/social.npy')

In [0]:
N = order.shape[0]
iters = order.shape[1]

In [0]:
X1 = []
X2 = []
Y = []

for t in range(iters):
    
    states = np.zeros(N)
    
    
    for i in range(N):
        j = int(order[i,t])
        depart = np.zeros(N)
        depart[j] = 1
        X1.append(states.copy())
        X2.append(distances[t])
        Y.append(depart)
        states[j]=1
        
    
    
X1 = np.array(X1)
X2 = np.array(X2)
Y = np.array(Y)

In [23]:
Y[1]

array([0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [24]:

w1 = tf.Variable(tf.random.normal(shape=(1, ), dtype=tf.float64))
b  = tf.Variable(tf.random.normal(shape=(1,), dtype=tf.float64))

array([0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [0]:
states = tf.convert_to_tensor(X1)
states = tf.expand_dims(states,-1)

dists = tf.convert_to_tensor(X2)

labels = tf.convert_to_tensor(Y)



In [0]:
D = w1*tf.exp(-dists)

In [0]:
logits = tf.matmul(D,states)+b

In [0]:

soft_denom = tf.reduce_sum(tf.where(states==0,tf.exp(logits),0),axis=1)
soft_denom = tf.expand_dims(soft_denom,-1)
softmax = tf.where(states==0,tf.exp(logits) / soft_denom ,0)

softmax = tf.squeeze(softmax)
bb = labels * tf.math.log(softmax+ 1e-10)
xent = -tf.reduce_sum(labels * tf.math.log(softmax + 1e-10), 1)

loss = tf.reduce_mean(xent)

In [100]:
loss

<tf.Tensor: id=338, shape=(), dtype=float64, numpy=1.635649876504253>

In [78]:
bb[1]

array([[0.11550068],
       [0.11383899],
       [0.14128396],
       [0.12870218],
       [0.11185025],
       [0.13632296],
       [0.11616523],
       [0.13633574],
       [0.        ]])

In [57]:
aa

array([[0.11550068],
       [0.11383899],
       [0.14128396],
       [0.12870218],
       [0.11185025],
       [0.13632296],
       [0.11616523],
       [0.13633574],
       [0.        ]])

In [58]:
dists

array([[0.        , 0.46518716, 0.67781613, 0.56436899, 0.41829771,
        0.34669432, 0.29267234, 0.27925955, 0.44884994],
       [0.46518716, 0.        , 1.08806745, 0.93590849, 0.04708439,
        0.81015801, 0.68673588, 0.6769249 , 0.42782326],
       [0.67781613, 1.08806745, 0.        , 0.18165458, 1.04305953,
        0.49076045, 0.40267227, 0.72051491, 0.79906245],
       [0.56436899, 0.93590849, 0.18165458, 0.        , 0.89253997,
        0.47524038, 0.27206278, 0.67336673, 0.62165055],
       [0.41829771, 0.04708439, 1.04305953, 0.89253997, 0.        ,
        0.76358053, 0.64126736, 0.63406523, 0.40283276],
       [0.34669432, 0.81015801, 0.49076045, 0.47524038, 0.76358053,
        0.        , 0.31850925, 0.23956262, 0.72730428],
       [0.29267234, 0.68673588, 0.40267227, 0.27206278, 0.64126736,
        0.31850925, 0.        , 0.44578711, 0.45729812],
       [0.27925955, 0.6769249 , 0.72051491, 0.67336673, 0.63406523,
        0.23956262, 0.44578711, 0.        , 0.72748588],
